# Exploring Manifold-Constrained Hyper-Connections (mHC)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bassrehab/mhc-visualizer/blob/main/notebook/mhc_exploration.ipynb)

This notebook demonstrates the key insight from DeepSeek's mHC paper:
**unconstrained residual mixing matrices cause signal explosion in deep networks,
while doubly stochastic constraints keep signals bounded.**

We'll implement Sinkhorn-Knopp projection and visualize why it matters.

**Paper:** https://arxiv.org/abs/2512.24880

**Author:** Subhadip Mitra (contact@subhadipmitra.com)

**Repository:** https://github.com/bassrehab/mhc-visualizer

## Setup

First, let's install and import the required packages.

In [ ]:
# Install dependencies (uncomment if running in Colab)
# !pip install -q numpy matplotlib

import numpy as np
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

print("Setup complete!")

## The Sinkhorn-Knopp Algorithm

The Sinkhorn-Knopp algorithm projects any positive matrix onto the set of **doubly stochastic matrices** - matrices where all rows and columns sum to 1.

### Why Doubly Stochastic?

Doubly stochastic matrices have a crucial property: **they are closed under multiplication**. This means:
- If A and B are doubly stochastic, then A @ B is also doubly stochastic
- The spectral norm is bounded: ||A|| ≤ 1
- Signal propagation stays bounded even through many layers

### The Algorithm

Starting from any positive matrix, we alternate between:
1. Normalizing rows to sum to 1
2. Normalizing columns to sum to 1

This converges to a doubly stochastic matrix!

In [ ]:
def sinkhorn_knopp(matrix: np.ndarray, iterations: int = 20, epsilon: float = 1e-8) -> np.ndarray:
    """
    Project a matrix onto the set of doubly stochastic matrices
    using the Sinkhorn-Knopp algorithm.
    
    Args:
        matrix: Input matrix (will be exponentiated to ensure positivity)
        iterations: Number of Sinkhorn iterations
        epsilon: Small value for numerical stability
        
    Returns:
        Doubly stochastic matrix (rows and columns sum to 1)
    """
    # Exponentiate to ensure all entries are positive
    P = np.exp(matrix)
    
    for _ in range(iterations):
        # Normalize rows
        P = P / (P.sum(axis=1, keepdims=True) + epsilon)
        # Normalize columns
        P = P / (P.sum(axis=0, keepdims=True) + epsilon)
    
    return P


def is_doubly_stochastic(matrix: np.ndarray, tol: float = 1e-6) -> Tuple[bool, float, float]:
    """
    Check if a matrix is doubly stochastic.
    
    Returns:
        (is_valid, row_error, col_error)
    """
    row_sums = matrix.sum(axis=1)
    col_sums = matrix.sum(axis=0)
    
    row_error = np.max(np.abs(row_sums - 1))
    col_error = np.max(np.abs(col_sums - 1))
    
    is_valid = row_error < tol and col_error < tol
    return is_valid, row_error, col_error


print("Sinkhorn-Knopp implementation ready!")

## Interactive Demo: Single Matrix Projection

Let's see how a random matrix becomes doubly stochastic through Sinkhorn projection.

In [ ]:
# Create a random matrix
n = 4
random_matrix = np.random.randn(n, n)

# Project onto doubly stochastic
projected = sinkhorn_knopp(random_matrix, iterations=20)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Original matrix
im1 = axes[0].imshow(random_matrix, cmap='RdBu', vmin=-2, vmax=2)
axes[0].set_title('Original Random Matrix')
axes[0].set_xlabel(f'Row sums: {random_matrix.sum(axis=1).round(2)}')
plt.colorbar(im1, ax=axes[0])

# Projected matrix
im2 = axes[1].imshow(projected, cmap='Blues', vmin=0, vmax=0.5)
axes[1].set_title('After Sinkhorn Projection (20 iters)')
axes[1].set_xlabel(f'Row sums: {projected.sum(axis=1).round(3)}')
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()

# Verify doubly stochastic property
is_valid, row_err, col_err = is_doubly_stochastic(projected)
print(f"\nIs doubly stochastic: {is_valid}")
print(f"Max row sum deviation: {row_err:.2e}")
print(f"Max col sum deviation: {col_err:.2e}")

## The Manifold Dial: Varying Sinkhorn Iterations

This is the key insight! Watch how the matrix transforms as we increase iterations:
- **0 iterations**: Just exp(M), unconstrained
- **2-5 iterations**: Partial projection, some structure
- **10-20 iterations**: Fully doubly stochastic

In [ ]:
# The "Manifold Dial" - sweep through Sinkhorn iterations
iterations_to_test = [0, 1, 2, 5, 10, 20]

fig, axes = plt.subplots(2, 3, figsize=(14, 9))
axes = axes.flatten()

# Use same random matrix for all
base_matrix = np.random.randn(4, 4)

for idx, iters in enumerate(iterations_to_test):
    if iters == 0:
        # Just exponentiate, no normalization
        result = np.exp(base_matrix)
    else:
        result = sinkhorn_knopp(base_matrix, iterations=iters)
    
    # Check properties
    is_valid, row_err, col_err = is_doubly_stochastic(result)
    max_val = result.max()
    
    im = axes[idx].imshow(result, cmap='Blues', vmin=0, vmax=max(0.5, result.max()))
    axes[idx].set_title(f'{iters} iterations')
    axes[idx].set_xlabel(f'Row err: {row_err:.2e}')
    plt.colorbar(im, ax=axes[idx])

plt.suptitle('The Manifold Dial: Sinkhorn Iterations Transform Random → Doubly Stochastic', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Stability Metrics

To understand why mHC is stable, we need to measure how matrices affect signal propagation:

- **Forward Gain**: Maximum row sum (how much a signal can be amplified)
- **Backward Gain**: Maximum column sum (gradient flow)
- **Spectral Norm**: Largest singular value (operator norm)

In [ ]:
def forward_gain(matrix: np.ndarray) -> float:
    """Maximum row sum - measures worst-case signal amplification."""
    return np.max(np.sum(np.abs(matrix), axis=1))


def backward_gain(matrix: np.ndarray) -> float:
    """Maximum column sum - measures worst-case gradient amplification."""
    return np.max(np.sum(np.abs(matrix), axis=0))


def spectral_norm(matrix: np.ndarray) -> float:
    """Largest singular value - the operator norm."""
    return np.linalg.svd(matrix, compute_uv=False)[0]


# Compare metrics for HC vs mHC
print("Comparing a single random matrix (HC) vs Sinkhorn-projected (mHC):\n")

# HC: Random matrix (exponentiated for fair comparison)
hc_matrix = np.exp(np.random.randn(4, 4))

# mHC: Sinkhorn-projected
mhc_matrix = sinkhorn_knopp(np.random.randn(4, 4), iterations=20)

print("HC (unconstrained):")
print(f"  Forward gain:  {forward_gain(hc_matrix):.4f}")
print(f"  Backward gain: {backward_gain(hc_matrix):.4f}")
print(f"  Spectral norm: {spectral_norm(hc_matrix):.4f}")

print("\nmHC (doubly stochastic):")
print(f"  Forward gain:  {forward_gain(mhc_matrix):.4f}")
print(f"  Backward gain: {backward_gain(mhc_matrix):.4f}")
print(f"  Spectral norm: {spectral_norm(mhc_matrix):.4f}")

## The Explosion Problem: Composite Mapping

The real issue isn't single-layer behavior—it's what happens when we **compose many layers**.

In a deep network, the effective transformation is:
$$H_{composite} = H_L \cdot H_{L-1} \cdot ... \cdot H_1$$

Let's simulate this for both HC and mHC.

In [ ]:
def generate_residual_matrix(n: int, method: str, sinkhorn_iters: int = 20) -> np.ndarray:
    """
    Generate a residual mixing matrix.
    
    Args:
        n: Matrix size
        method: 'baseline' (identity), 'hc' (random), or 'mhc' (Sinkhorn-projected)
        sinkhorn_iters: Iterations for mHC
    """
    if method == 'baseline':
        return np.eye(n)
    elif method == 'hc':
        # Random matrix - this is the problem!
        return np.random.randn(n, n)
    elif method == 'mhc':
        # Sinkhorn-projected - this is the solution!
        return sinkhorn_knopp(np.random.randn(n, n), iterations=sinkhorn_iters)
    else:
        raise ValueError(f"Unknown method: {method}")


def simulate_depth(depth: int, n: int, method: str, sinkhorn_iters: int = 20) -> List[float]:
    """
    Simulate composite forward gain over depth.
    
    Returns list of forward gains at each layer.
    """
    composite = np.eye(n)
    gains = []
    
    for _ in range(depth):
        H = generate_residual_matrix(n, method, sinkhorn_iters)
        composite = H @ composite
        gains.append(forward_gain(composite))
    
    return gains


print("Simulation functions ready!")

In [ ]:
# Simulate composite mapping explosion
depth = 64
n = 4

# Set seed for reproducibility
np.random.seed(42)
baseline_gains = simulate_depth(depth, n, 'baseline')

np.random.seed(42)
hc_gains = simulate_depth(depth, n, 'hc')

np.random.seed(42)
mhc_gains = simulate_depth(depth, n, 'mhc', sinkhorn_iters=20)

# Plot!
plt.figure(figsize=(12, 6))
plt.semilogy(baseline_gains, 'g-', linewidth=2, label='Baseline (Identity)', alpha=0.8)
plt.semilogy(hc_gains, 'r-', linewidth=2, label='HC (Random)', alpha=0.8)
plt.semilogy(mhc_gains, 'b-', linewidth=2, label='mHC (Sinkhorn)', alpha=0.8)

plt.xlabel('Layer Depth', fontsize=12)
plt.ylabel('Composite Forward Gain (log scale)', fontsize=12)
plt.title('Signal Explosion: HC vs mHC Composite Mapping', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nFinal forward gains at depth {depth}:")
print(f"  Baseline: {baseline_gains[-1]:.2f}")
print(f"  HC:       {hc_gains[-1]:.2e}  ← EXPLOSION!")
print(f"  mHC:      {mhc_gains[-1]:.2f}  ← Stable!")

## The Money Plot: Recreating Figure 3

This is the key visualization from the paper, showing why mHC matters for deep networks.

In [ ]:
# High-quality plot for publication/sharing
fig, ax = plt.subplots(figsize=(10, 6), dpi=150)

# Re-run with fresh seeds
depth = 64

np.random.seed(42)
baseline = simulate_depth(depth, 4, 'baseline')

np.random.seed(42) 
hc = simulate_depth(depth, 4, 'hc')

np.random.seed(42)
mhc = simulate_depth(depth, 4, 'mhc', sinkhorn_iters=20)

layers = range(1, depth + 1)

ax.semilogy(layers, baseline, color='#10b981', linewidth=2.5, label='Baseline (Identity)', alpha=0.9)
ax.semilogy(layers, hc, color='#ef4444', linewidth=2.5, label='HC (Unconstrained)', alpha=0.9)
ax.semilogy(layers, mhc, color='#3b82f6', linewidth=2.5, label='mHC (Sinkhorn k=20)', alpha=0.9)

# Styling
ax.set_xlabel('Layer Depth', fontsize=13)
ax.set_ylabel('Composite Forward Gain', fontsize=13)
ax.set_title('The Manifold Dial: HC Explosion vs mHC Stability', fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(1, depth)

# Add annotation
ax.annotate(f'HC: {hc[-1]:.1e}', xy=(depth, hc[-1]), xytext=(depth-15, hc[-1]*10),
            fontsize=10, color='#ef4444',
            arrowprops=dict(arrowstyle='->', color='#ef4444', alpha=0.7))

ax.annotate(f'mHC: {mhc[-1]:.2f}', xy=(depth, mhc[-1]), xytext=(depth-15, mhc[-1]*0.1),
            fontsize=10, color='#3b82f6',
            arrowprops=dict(arrowstyle='->', color='#3b82f6', alpha=0.7))

plt.tight_layout()
plt.savefig('mhc_hero_plot.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print("\nPlot saved as 'mhc_hero_plot.png'")

## Why Does This Work?

### The Mathematics of Stability

Doubly stochastic matrices have three key properties:

1. **Spectral norm ≤ 1**: The matrix doesn't amplify signals
2. **Closed under multiplication**: Products remain doubly stochastic  
3. **Convex combinations of permutations**: Acts like a weighted average (Birkhoff-von Neumann)

When you multiply many doubly stochastic matrices together, the result stays bounded because each multiplication is **non-expansive**.

In contrast, unconstrained matrices compound their gains exponentially:
- If each matrix has gain 1.1, after 64 layers: 1.1^64 ≈ 300
- If each matrix has gain 1.5, after 64 layers: 1.5^64 ≈ 10^11 (!)

In [ ]:
# Demonstrate the mathematical property
print("Demonstrating doubly stochastic closure under multiplication:\n")

# Generate two doubly stochastic matrices
A = sinkhorn_knopp(np.random.randn(4, 4), iterations=20)
B = sinkhorn_knopp(np.random.randn(4, 4), iterations=20)

# Their product
C = A @ B

print("Matrix A:")
print(f"  Row sums: {A.sum(axis=1).round(4)}")
print(f"  Col sums: {A.sum(axis=0).round(4)}")
print(f"  Spectral norm: {spectral_norm(A):.4f}")

print("\nMatrix B:")
print(f"  Row sums: {B.sum(axis=1).round(4)}")
print(f"  Col sums: {B.sum(axis=0).round(4)}")
print(f"  Spectral norm: {spectral_norm(B):.4f}")

print("\nProduct C = A @ B:")
print(f"  Row sums: {C.sum(axis=1).round(4)}")
print(f"  Col sums: {C.sum(axis=0).round(4)}")
print(f"  Spectral norm: {spectral_norm(C):.4f}")
print("\n→ Product is also doubly stochastic! Spectral norm bounded!")

## The Manifold Dial: Interactive Experiment

Let's see how varying Sinkhorn iterations affects the final composite gain.

In [ ]:
# Sweep Sinkhorn iterations and see the effect
sinkhorn_values = [0, 1, 2, 3, 5, 10, 15, 20, 25, 30]
final_gains = []

depth = 64
n = 4

for k in sinkhorn_values:
    np.random.seed(42)  # Same seed for fair comparison
    
    if k == 0:
        # k=0 means no projection, like HC
        gains = simulate_depth(depth, n, 'hc')
    else:
        gains = simulate_depth(depth, n, 'mhc', sinkhorn_iters=k)
    
    final_gains.append(gains[-1])

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

ax.semilogy(sinkhorn_values, final_gains, 'bo-', linewidth=2, markersize=8)
ax.axhline(y=1, color='g', linestyle='--', alpha=0.5, label='Ideal (gain=1)')
ax.axhline(y=2, color='orange', linestyle='--', alpha=0.5, label='Stable threshold')

ax.set_xlabel('Sinkhorn Iterations (k)', fontsize=13)
ax.set_ylabel('Final Composite Gain at Depth 64', fontsize=13)
ax.set_title('The Manifold Dial: Stability vs Sinkhorn Iterations', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nSinkhorn Iterations → Final Gain:")
for k, g in zip(sinkhorn_values, final_gains):
    status = "UNSTABLE" if g > 10 else "stable" if g < 2 else "marginal"
    print(f"  k={k:2d}: {g:12.2e}  ({status})")

## Key Takeaways

1. **HC (Hyper-Connections)** use unconstrained residual mixing matrices
   - Each layer's matrix can have arbitrary row/column sums
   - Over many layers, these compound → **exponential explosion**

2. **mHC (Manifold-Constrained HC)** projects matrices onto the Birkhoff polytope
   - Uses Sinkhorn-Knopp to ensure doubly stochastic matrices
   - Spectral norm ≤ 1, so products stay bounded → **stable signals**

3. **The "Manifold Dial"** is the Sinkhorn iteration count (k)
   - k=0: Unconstrained (like HC) → unstable
   - k≥10: Well-projected → stable
   - Sweet spot around k=20 for most applications

---

**Try it yourself!** Modify the parameters above and re-run to explore:
- Different depths (try 100, 200)
- Different matrix sizes (n=2, 8, 16)
- Different random seeds

**Interactive Demo:** https://github.com/bassrehab/mhc-visualizer

---

## References

- **mHC Paper**: [DeepSeek-AI, arXiv:2512.24880](https://arxiv.org/abs/2512.24880)
- **Sinkhorn-Knopp Algorithm**: Sinkhorn & Knopp (1967)
- **This Notebook**: [github.com/bassrehab/mhc-visualizer](https://github.com/bassrehab/mhc-visualizer)

**Author**: Subhadip Mitra (contact@subhadipmitra.com)